## Start

In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)



___
### Tags: #Source #Mobile

### Links: 
___

In [ ]:
# creating a table from s3

query_text = """--sql
    CREATE TABLE db1.subscriptions_st_mobile
(
    `report_date` Date,
    `citizen_id` Int32,
    `state` String,
    `created_at` String,
    `subscribed_from` String,
    `auto_renew_status` Int16,
    `activated_at` String,
    `plan` String,
    `expires_date` String,
    `renew_stopped_at` String,
    `renew_failed_at` String,
    `started_from` String,
    `renew_fail_reason` String
)
    ENGINE = S3('https://storage.yandexcloud.net/dwh-asgard/subscriptions_st_mobile/year=*/month=*/*.csv','CSVWithNames');
    """

ch.query_run(query_text)


In [ ]:
# creating a table for materialized view

query_text = """--sql
    CREATE TABLE db1.subscriptions_st_mobile_ch
    (
    `report_date` Date,
    `citizen_id` Int32,
    `state` String,
    `created_at` String,
    `subscribed_from` String,
    `auto_renew_status` Int16,
    `activated_at` String,
    `plan` String,
    `expires_date` String,
    `renew_stopped_at` String,
    `renew_failed_at` String,
    `started_from` String,
    `renew_fail_reason` String
    )
    ENGINE = MergeTree()
    ORDER BY citizen_id
    """

ch.query_run(query_text)

In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.subscriptions_st_mobile_mv
    REFRESH EVERY 1 DAY OFFSET 3 HOUR RANDOMIZE FOR 1 HOUR TO db1.subscriptions_st_mobile_ch AS
    SELECT
        *
    FROM db1.subscriptions_st_mobile
    """

ch.query_run(query_text)

___
## Tools
___
### query

In [2]:
query_text = """--sql
    SELECT
        *
    FROM db1.subscriptions_st_mobile_ch
    ORDER BY report_date DESC
    LIMIT 10
    """

ch.query_run(query_text)

,report_date,citizen_id,state,created_at,subscribed_from,auto_renew_status,activated_at,plan,expires_date,renew_stopped_at,renew_failed_at,started_from,renew_fail_reason
0,2025-09-14,139,deactivated,2021-02-18 17:11:33,yakassa,0,,face_rec_p1m,2021-04-18 17:15:26,,,,
1,2025-09-14,51,deactivated,2025-07-29 10:15:16,yakassa,0,,standard_p1m,,,,hms,
2,2025-09-14,829,deactivated,2022-10-07 18:12:05,ios,0,2023-06-07 18:15:04,standard_p1m,2023-07-07 18:11:55,,,,
3,2025-09-14,273,deactivated,2023-04-27 18:09:32,yakassa,0,2025-03-24 17:46:12,standard_p1m,,,2025-05-01 18:55:35,,no_payment_method
4,2025-09-14,202,deactivated,2024-05-23 10:52:22,ios,0,2024-09-09 13:32:57,standard_p1m,,,,ios,
5,2025-09-14,15,activated,2024-10-02 08:34:16,yakassa,1,2025-09-02 08:35:12,standard_p1m,2025-10-02 08:34:30,,,ios,
6,2025-09-14,111,deactivated,2023-08-19 09:48:38,yakassa,0,2023-11-05 18:16:24,standard_p1m,2023-12-05 18:16:24,,,,
7,2025-09-14,108,activated,2025-04-21 17:51:08,yakassa,1,2025-08-21 17:55:14,standard_p1m_t82,2025-09-21 17:51:13,,,ios,
8,2025-09-14,799,deactivated,2025-03-26 21:57:19,yakassa,0,,standard_p1m,,,,hms,
9,2025-09-14,108,deactivated,2023-08-24 15:47:27,yakassa,0,2025-01-21 14:40:17,standard_p1m,,,2025-02-28 15:51:00,ios,insufficient_funds


In [2]:
query_text = """--sql
    SELECT
    DISTINCT
        state
    FROM db1.subscriptions_st_mobile_ch
    
    """

ch.query_run(query_text)

state
str
"""deactivated"""
"""activated"""
"""payed"""
"""waiting"""


### Drop ch

In [ ]:
query_text = """
    DROP TABLE db1.subscriptions_st_mobile_ch
    """
ch.query_run(query_text)

### Drop mv

In [3]:
query_text = """
    DROP TABLE db1.subscriptions_st_mobile_mv
    """
ch.query_run(query_text)

,read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
0,0,0,0,0,0,0,0,1292353,8b968d23-fe3f-4647-a4e2-90af3d2193a7
